# Metric to Characterize Local Ordinance Restrictiveness
Regulations vary widely in their restrictiveness. Regulations are also complex and multidimensional - they include various setbacks, height limits, noise limits, shading limits, etc. We would like a summary way to compare them without needing to reckon with all the details.

In [1]:
import dbcp
import pandas as pd
import numpy as np

/app/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
engine = dbcp.helpers.get_sql_engine()

In [3]:
nrel = pd.read_sql('select * from data_warehouse.nrel_local_ordinances', con=engine)

In [4]:
nrel.head(2)

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
0,Alabama,None,Baldwin,Banned,None,None,Baldwin County Zoning Ordinance Sec. 13.13,None,None,NaN,None,NaN,2018.0,NaN,No Change,banned,None,NaN,wind,01,01003,Baldwin County,county,Baldwin County
1,Alabama,None,Cherokee,Sound,dBA,40,Ala. Code § 45-10-260.05,None,None,NaN,None,NaN,2021.0,NaN,None,sound,dba,40.0,wind,01,01019,Cherokee County,county,Cherokee County


In [122]:
nrel.shape

(2671, 29)

In [123]:
nrel.drop_duplicates(subset=['county_id_fips', 'raw_town_name']).shape

(707, 29)

In [5]:
nrel.columns

Index(['raw_state_name', 'raw_town_name', 'raw_county_name', 'raw_ordinance_type', 'raw_units', 'raw_value', 'raw_citation', 'raw_comment', 'raw_updated_unit', 'raw_updated_value', 'raw_updated_comment', 'year_enacted', 'year_recorded', 'updated_year_recorded', 'update_status', 'ordinance_type', 'units', 'value', 'energy_type', 'state_id_fips', 'county_id_fips', 'geocoded_locality_name', 'geocoded_locality_type', 'geocoded_containing_county'], dtype='object')

In [11]:
pd.options.display.max_rows = 100

In [47]:
ordinance_mix = nrel.groupby(['energy_type', 'ordinance_type', 'units'], dropna=False)['value'].agg(['size', 'describe'])

In [53]:
ordinance_mix.columns = ordinance_mix.columns.droplevel(0)
ordinance_mix.rename(columns={'value': 'n', 'count': 'n_not_null'}, inplace=True)

In [56]:
# arrange by most frequent ordinance types within each energy type
sort_order = (ordinance_mix.groupby(level=['energy_type', 'ordinance_type'])['n']
              .sum()
              .reset_index(drop=False)
              .sort_values(['energy_type', 'n'])
              .set_index(['energy_type', 'ordinance_type'])
             )

In [57]:
sort_order['sort_order'] = range(len(sort_order))

In [58]:
ordinance_mix = ordinance_mix.join(sort_order['sort_order']).sort_values(['sort_order', 'n'], ascending=False)
ordinance_mix

n  n_not_null         mean          std          min          25%          50%          75%          max  sort_order
energy_type ordinance_type          units                                                                                                                                                      
wind        structures              meters                               229       229.0   499.239724   456.780244     3.000000   304.799988   305.000000   609.000000  3218.689941          32
                                    max tip height multiplier            136       136.0    13.586250    65.892623     1.000000     1.175000     2.000000     3.000000   457.200012          32
                                    rotor diameter multiplier              5         5.0     4.400000     1.635543     1.500000     5.000000     5.000000     5.000000     5.500000          32
                                    hub height multiplier                  2         2.0     3.250000     2.474874     1.500000     2.375000     3.250000     4.125000     5.000000          32
            property line           max tip height multiplier            253       253.0     1.464032     0.825523     0.500000     1.100000     1.100000     1.500000     6.500000          31
                                    meters                                68        68.0   515.984081   801.760188    45.720001   152.399994   273.919998   537.210022  4828.029785          31
                                    rotor diameter multiplier             28        28.0     1.664286     1.208064     0.500000     1.000000     1.050000     2.500000     5.000000          31
                                    hub height multiplier                  4         4.0     1.587500     0.947255     1.000000     1.075000     1.175000     1.687500     3.000000          31
                                    rotor radius multiplier                1         1.0     1.000000          NaN     1.000000     1.000000     1.000000     1.000000     1.000000          31
            roads                   max tip height multiplier            270       270.0     1.299259     0.562720     1.000000     1.100000     1.100000     1.500000     6.500000          30
                                    meters                                60        60.0   369.774209   759.983129    45.720001    91.440002   160.144005   258.555000  4828.029785          30
                                    rotor diameter multiplier             12        12.0     1.125000     1.316417     0.500000     0.500000     0.500000     1.125000     5.000000          30
                                    hub height multiplier                  2         2.0     1.300000     0.282843     1.100000     1.200000     1.300000     1.400000     1.500000          30
            sound                   dba                                  218       218.0    52.371560     7.876613    30.000000    50.000000    52.500000    60.000000    70.000000          29
                                    max tip height multiplier              1         1.0    55.000000          NaN    55.000000    55.000000    55.000000    55.000000    55.000000          29
                                    rotor diameter multiplier              1         1.0    50.000000          NaN    50.000000    50.000000    50.000000    50.000000    50.000000          29
            transmission            max tip height multiplier            161       161.0     1.319565     0.698596     1.000000     1.100000     1.100000     1.250000     6.500000          28
                                    meters                                19        19.0   343.464206   720.640533    60.959999    60.959999   163.190002   222.759995  3218.689941          28
                                    rotor diameter multiplier              2         2.0     3.250000     2.474874     1.500000     2.375000     3.250000     4.125000     5.000000          28
            height                  meters 

In [149]:
ordinance_mix.shape

(63, 10)

In [63]:
# erroneous units - these should be dBA
nrel.query("ordinance_type == 'sound' and units.str.contains('multiplier')")

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
1650,Tennessee,None,Shelby,Sound,Max tip-height Multiplier,55,Shelby County Unified Dev. Code § 2.62(J),None,None,NaN,None,NaN,2022.0,NaN,None,sound,max tip height multiplier,55.0,wind,47,47157,Shelby County,county,Shelby County
1808,Wyoming,None,Laramie,Sound,Rotor-Diameter Multiplier,50,2-2-125 Large Wind Energy Systems,None,None,NaN,None,NaN,2022.0,NaN,None,sound,rotor diameter multiplier,50.0,wind,56,56021,Laramie County,county,Laramie County


### Simplify Categories
To facilitate comparison, we'll start with the most common ordinance types plus outright bans.
#### Wind:
* structure setbacks
* property line setbacks
* road setbacks (maybe combine with highways?)
* sound constraints
* transmission setbacks
* height limits
* bans

Forget about:
* water setbacks
* railroad setbacks
* density and lot size limitations
* shadow flicker
* pipeline setbacks
* local features like ridgelines or the appalachian trail

Undecided: county level caps are basically conditional bans
* county capacity caps
* county turbine caps

#### Solar:
* property line setbacks
* height limits
* structure setbacks
* road setbacks (maybe combine with highways)

Forget about:
* lot size limts
* sound constraints
* water setbacks
* density/coverage limits
* railroad setbacks
* max project size
* max project capacity

## Implementation
### Standardize Setback Distances
Setback distances can be defined in various ways: by hub height multipliers, tip height multipliers, rotor diameter multipliers, simple distances, etc. We can convert all of those to simple distances with a few assumptions about common heights to plug into those multipliers.

The most common turbine configuration in 2020 and 2021, which was >3 times more common than the second most common, was a 89 m hub height, 127 m rotor diameter turbine with a combined height of 152.4 m.
* tip height: 500 feet (152.4 meters), because that is the soft cap set by FAA requirements. 65% of 2021 turbines are at or under this height per USGS turbine database
* hub height: 89 meters (median and mode of 2021 turbines)
* rotor diameter: 127 m (median and mode of 2021 turbines)

Solar project height is less documented, but there are only 10 regulations based on solar project height. I'll plan to use 20 feet because it is a common height limit.

In [98]:
hub_height = 89.
rotor_diameter = 127.
tip_height = 152.4
structure_height = 20 * 12 * 2.54 / 100  # 20 feet to meters

In [92]:
nrel.loc[nrel['units'].isna(), 'raw_ordinance_type'].str.strip().value_counts(dropna=False)

Banned        56
Moratorium     3
Name: raw_ordinance_type, dtype: int64

In [95]:
nrel['units'] = nrel['units'].replace({'maximum structure height': 'maximum structure height multiplier'}) # all between 1 and 2; clearly multipliers

In [96]:
is_multiplier = nrel['units'].str.contains('multiplier').fillna(False)

In [97]:
nrel.loc[is_multiplier, 'units'].value_counts(dropna=False)

max tip height multiplier              908
rotor diameter multiplier               58
maximum structure height multiplier     10
hub height multiplier                    8
rotor radius multiplier                  1
Name: units, dtype: int64

In [100]:
multiplier_mapping = {
    "hub height": hub_height,
    "tip height": tip_height,
    "structure height": structure_height,
    "rotor radius": rotor_diameter / 2,
    "rotor diameter": rotor_diameter,
}

In [104]:
nrel['multiplier_height'] = 1
nrel['standardized_units'] = nrel['units'].copy()
for term, value in multiplier_mapping.items():
    _filter = nrel['units'].str.contains(term, regex=False).fillna(False) & is_multiplier
    nrel.loc[_filter, 'multiplier_height'] = value
    nrel.loc[_filter, 'standardized_units'] = 'meters'

In [103]:
nrel['standardized_value'] = nrel['value'] * nrel['multiplier_height']

In [105]:
standardized_ordinance_mix = nrel.groupby(['energy_type', 'ordinance_type', 'standardized_units'], dropna=False)['standardized_value'].agg(['size', 'describe'])

In [108]:
standardized_ordinance_mix.columns = standardized_ordinance_mix.columns.droplevel(0)
standardized_ordinance_mix.rename(columns={'standardized_value': 'n', 'count': 'n_not_null'}, inplace=True)

In [ ]:
standardized_ordinance_mix = standardized_ordinance_mix.join(sort_order['sort_order']).sort_values(['sort_order', 'n'], ascending=False)

In [119]:
standardized_ordinance_mix.drop(columns=['n_not_null', 'sort_order']).reset_index().to_csv('/app/data/output/nrel_ordinance_summary.csv')

### Distinguish Structure Types
The NREL database relies on text comments to differentiate between participating/nonparticipating and residences/other buildings. The same literal distance is much more restrictive if applied to participating residences than to all nonparticipating structures. Nonparticipating setbacks are much less important because those structures already sit behind property line setbacks in many cases.

In [82]:
nrel['is_offsite'] = nrel['raw_comment'].str.lower().str.contains(r'off[- ]?site|non-?participat', regex=True)
nrel['is_residence'] = nrel['raw_comment'].str.lower().str.contains(r'dwelling|residen|inhabit|habitable', regex=True)

In [83]:
nrel.query("energy_type == 'wind' and ordinance_type == 'structures' and raw_comment.notnull()").sort_values('county_id_fips')

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,is_offsite,is_residence
11,Arizona,None,Apache,Structures,Meters,749.808,Apache County Art. 750,1/2 mile (2460 feet) from offsite residences,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,749.807983,wind,04,04001,Apache County,county,Apache County,True,True
22,Arizona,None,Cochise,Structures,Meters,304.8,Cochise County Ord. § 1822.02,Minimum 1000 feet setback from all residences,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,304.799988,wind,04,04003,Cochise County,county,Cochise County,False,True
26,Arizona,None,Navajo,Structures,Meters,782,"Navajo County Ord. Art 20, 6-10(4)(w)",Setback is minimum half mile (2640 Feet) to residence off property line,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,782.000000,wind,04,04017,Navajo County,county,Navajo County,False,True
29,California,None,Butte,Structures,Meters,305,Butte County Code § 24-157(E)(2),1000 feet from offsite residences,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,305.000000,wind,06,06007,Butte County,county,Butte County,True,True
32,California,None,Contra Costa,Structures,Meters,305,Contra Costa County Code Title 8 § 88-3,1000 feet from offsite residences,None,NaN,None,NaN,2018.0,2021.0,No Change,structures,meters,305.000000,wind,06,06013,Contra Costa County,county,Contra Costa County,True,True
36,California,None,Glenn,Structures,Meters,61,Glenn County Code § 15.860.030(G),200 Feet from Residences off property,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,61.000000,wind,06,06021,Glenn County,county,Glenn County,False,True
40,California,None,Kern,Structures,Max tip-height Multiplier,1.5,Kern County Code § 19.64.140(F),Greater of 1.5x max tip height or 500 feet to offsite residences,None,NaN,None,NaN,NaN,NaN,None,structures,max tip height multiplier,1.500000,wind,06,06029,Kern County,county,Kern County,True,True
57,California,None,Monterey,Structures,Max tip-height Multiplier,1.25,Monterey County Code §21.64.120,1.25x height habitable structure,None,NaN,None,NaN,2018.0,2021.0,No Change,structures,max tip height multiplier,1.250000,wind,06,06053,Monterey County,county,Monterey County,False,True
87,California,None,Solano,Structures,Meters,3,Solano County Code § 28.80,Ten foot setback from other structures on the property,None,NaN,None,NaN,2021.0,NaN,None,structures,meters,3.000000,wind,06,06095,Solano County,county,Solano County,False,False
78,California,None,Sonoma,Structures,Max tip-height Multiplier,1,Sonoma County Code § 26-88-208(d),Minimum setback of 100% the tower height to structure offsite,None,NaN,None,NaN,2021.0,NaN,None,structures,max tip height multiplier,1.000000,wind,06,06097,Sonoma County,county,Sonoma County,True,False


### [Future Work] Standardize Multi-valued Setbacks
Some property line setbacks are of the general form "the greater/lesser of 1.5x tip height or 500 feet". This seems straightforward to resolve with the same height assumptions used above. But there are only 18 of them so I'll defer for now.

In [113]:
nrel.query("ordinance_type == 'property line'")['raw_comment'].str.lower().str.contains('lesser|greater').fillna(False).agg(['sum', 'mean'])

sum     18.000000
mean     0.030664
Name: raw_comment, dtype: float64

In [110]:
nrel.query("energy_type == 'wind' and ordinance_type == 'property line' and raw_comment.notnull()").sort_values('county_id_fips')

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,is_offsite,is_residence,multiplier_height,standardized_value,standardized_units
23,Arizona,None,Cochise,Property Line,Max tip-height Multiplier,1 + 10 ft,Cochise County Ord. § 1822.02,Setback is max tip-height plus ten feet (Some percentage above 100%),None,NaN,None,NaN,2021.0,NaN,None,property line,meters,154.048004,wind,04,04003,Cochise County,county,Cochise County,False,False,1.0,154.048004,meters
25,Arizona,None,La Paz,Property Line,Max tip-height Multiplier,1.01,La Paz County Ord. § 619.06,Setback is minimum 101% of max tip height to the property line,None,NaN,None,NaN,2021.0,NaN,None,property line,max tip height multiplier,1.010000,wind,04,04012,La Paz County,county,La Paz County,False,False,152.4,153.923999,meters
31,California,None,Contra Costa,Property Line,Hub-height Multiplier,3,Contra Costa County Code Title 8 § 88-3,Greater of 3x tower height or 500 feet from boundary line and right-of-way,None,NaN,None,NaN,2018.0,2021.0,No Change,property line,hub height multiplier,3.000000,wind,06,06013,Contra Costa County,county,Contra Costa County,False,False,89.0,267.000000,meters
38,California,None,Kern,Property Line,Max tip-height Multiplier,2,Kern County Code § 19.64.140(F),lesser of 2x max tip height or 500 feet to property line unless owned by same project developer if project is less than 40 acres in size,None,NaN,None,NaN,NaN,NaN,None,property line,max tip height multiplier,2.000000,wind,06,06029,Kern County,county,Kern County,False,False,152.4,304.800000,meters
39,California,None,Kern,Property Line,Max tip-height Multiplier,1.5,Kern County Code § 19.64.140(F),lesser of 1.5x max tip height or 500 feet to property line unless owned by same project developer if project is more than 40 acres in size,None,NaN,None,NaN,2018.0,2021.0,No Change,property line,max tip height multiplier,1.500000,wind,06,06029,Kern County,county,Kern County,False,False,152.4,228.600000,meters
66,California,None,San Bernardino,Property Line,Max tip-height Multiplier,2,San Bernadino County Dev. Code § 84.29.030,"Setback of 2x max tip height if project is on less than 40 acres, waived if common property line is shared by project owner",None,NaN,None,NaN,NaN,2022.0,None,property line,max tip height multiplier,2.000000,wind,06,06071,San Bernardino County,county,San Bernardino County,False,False,152.4,304.800000,meters
67,California,None,San Bernardino,Property Line,Max tip-height Multiplier,1.5,San Bernadino County Dev. Code § 84.29.030,"Setback of 1.5x max tip height if project is on more than 40 acres, waived if common property line is shared by project owner",None,NaN,None,NaN,NaN,2022.0,None,property line,max tip height multiplier,1.500000,wind,06,06071,San Bernardino County,county,San Bernardino County,False,False,152.4,228.600000,meters
71,California,None,San Luis Obispo,Property Line,Max tip-height Multiplier,3,San Luis Obispo County Code § 22.32.060(b),3x Max height setback,None,NaN,None,NaN,2021.0,NaN,None,property line,max tip height multiplier,3.000000,wind,06,06079,San Luis Obispo County,county,San Luis Obispo County,False,False,152.4,457.200000,meters
84,California,None,Santa Clara,Property Line,Max tip-height Multiplier,1,Santa Clara County Code § 4.10.390,Setback equal to max tip height,None,NaN,None,NaN,2021.0,NaN,None,property line,max tip height multiplier,1.000000,wind,06,06085,Santa Clara County,county,Santa Clara County,False,False,152.4,152.400000,meters
86,California,None,Santa Cruz,Property Line,Max tip-height Multiplier,1,Santa Cruz County Code 12.24.060(I),Setback at least equal to max tip height,None,NaN,None,NaN,2021.0,NaN,None,property line,max tip height multi

In [121]:
nrel.query("energy_type == 'wind' and ordinance_type == 'total turbines'").sort_values('county_id_fips')

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,is_offsite,is_residence,multiplier_height,standardized_value,standardized_units
510,Iowa,None,Adair,Total Installation,Turbines,535,Adair County Ord. 31B § 13(b),Adair County caps the allowable number of turbines in the county at 535 (effective ban),None,NaN,None,NaN,2021.0,NaN,None,total turbines,turbines,535.0,wind,19,19001,Adair County,county,Adair County,False,False,1.0,535.0,turbines
602,Iowa,None,Fremont,Total Installation,Turbines,150,Fremont County Ord. 2020-1 § 5(E),Fremont County caps the number of turbines allowed at 150,None,NaN,None,NaN,2021.0,NaN,None,total turbines,turbines,150.0,wind,19,19071,Fremont County,county,Fremont County,False,False,1.0,150.0,turbines
664,Iowa,None,Madison,Total Installation,Turbines,51,Madison County Ord. 54.40.2,Effective ban on wind due to cap of 51 total turbines,None,NaN,None,NaN,2021.0,NaN,None,total turbines,turbines,51.0,wind,19,19121,Madison County,county,Madison County,False,False,1.0,51.0,turbines


### Drop overly-local geographies
Rationale: nobody is trying to build utility scale energy projects downtown

In [129]:
nrel.drop_duplicates(subset=['geocoded_locality_name', 'county_id_fips'])['geocoded_locality_type'].value_counts(dropna=False)

county    664
city       40
town        2
            1
Name: geocoded_locality_type, dtype: int64

In [132]:
counties = pd.Series(nrel.query("geocoded_locality_type == 'county'")['county_id_fips'].unique())
len(counties)

663

In [147]:
random_county = counties.sample(random_state=4).iat[0]
random_county

'17147'

In [148]:
nrel.query("county_id_fips == @random_county")

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,is_offsite,is_residence,multiplier_height,standardized_value,standardized_units
235,Illinois,None,Piatt,Transmission,Max tip-height Multiplier,1.1,Piatt County Zoning Ord. Appdx. A § 7,None,None,NaN,None,NaN,2021.0,NaN,None,transmission,max tip height multiplier,1.10,wind,17,17147,Piatt County,county,Piatt County,None,None,152.4,167.640004,meters
302,Illinois,None,Piatt,Structures,Max tip-height Multiplier,1.3,Piatt County Zoning Ord. Appdx. A § 7,Greater of 1.3 max tip height or 1600 feet,None,NaN,None,NaN,2021.0,NaN,None,structures,max tip height multiplier,1.30,wind,17,17147,Piatt County,county,Piatt County,False,False,152.4,198.119993,meters
303,Illinois,None,Piatt,Property Line,Max tip-height Multiplier,1.3,Piatt County Zoning Ord. Appdx. A § 7,Greater of 1.3 max tip height or 1000 feet,None,NaN,None,NaN,2021.0,NaN,None,property line,max tip height multiplier,1.30,wind,17,17147,Piatt County,county,Piatt County,False,False,152.4,198.119993,meters
304,Illinois,None,Piatt,Height,Meters,190.5,Piatt County Zoning Ord. Appdx. A § 7,625 Foot Max Height,None,NaN,None,NaN,2021.0,NaN,None,height,meters,190.50,wind,17,17147,Piatt County,county,Piatt County,False,False,1.0,190.500000,meters
305,Illinois,None,Piatt,Roads,Max tip-height Multiplier,1.1,Piatt County Zoning Ord. Appdx. A § 7,None,None,NaN,None,NaN,2021.0,NaN,None,roads,max tip height multiplier,1.10,wind,17,17147,Piatt County,county,Piatt County,None,None,152.4,167.640004,meters
2026,Illinois,None,Piatt,Property Line,Meters,30.48,Piatt County Zoning Ord. Art. X(2)(a),None,None,NaN,None,NaN,2021.0,NaN,None,property line,meters,30.48,solar,17,17147,Piatt County,county,Piatt County,None,None,1.0,30.480000,meters


## Check Updated Values

In [8]:
nrel['raw_updated_value'].notnull().mean()

0.0041183077499064025

In [14]:
nrel['raw_updated_comment'].notnull().mean()

0.004492699363534257

In [9]:
pd.options.display.max_colwidth = None

In [15]:
# updates look like they have already been applied, with one exception (Jay County)
nrel[nrel['raw_updated_value'].notnull() | nrel['raw_updated_comment'].notnull()]

,raw_state_name,raw_town_name,raw_county_name,raw_ordinance_type,raw_units,raw_value,raw_citation,raw_comment,raw_updated_unit,raw_updated_value,raw_updated_comment,year_enacted,year_recorded,updated_year_recorded,update_status,ordinance_type,units,value,energy_type,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
361,Indiana,None,Clinton,Structures,Max tip-height Multiplier,1.1,Clinton County Wind Energy Ordinance Art 5 §535,None,Meters,1.10,Changed from 305 Meters to 1.1 Maxtip height multiplier for structure setbacks,2009.0,2018.0,2022.0,Change from meters to multiplier,structures,max tip height multiplier,1.100000,wind,18,18023,Clinton County,county,Clinton County
370,Indiana,None,Grant,Rivers,Meters,1011.94,Grant County Zoning Ordinance § 153.550,None,None,1011.94,Changed from 805 meters to 1011.94 meters,2008.0,2018.0,2022.0,Changed,water,meters,1011.940002,wind,18,18053,Grant County,county,Grant County
400,Indiana,None,Jay,Structures,Meters,305,Jay County Zoning Ordinace Sec. 217&218,None,Meters,457.20,Changed from 305 Meters to 457.2 Meters,NaN,2018.0,2022.0,Changed,structures,meters,305.000000,wind,18,18075,Jay County,county,Jay County
477,Indiana,None,Tipton,Structures,Meters,804.67,Tipton County Zoning Ordinance Art 5.,None,Meters,804.67,Changed from 305 meters to 804.67 meters,NaN,2018.0,2022.0,Changed,structures,meters,804.669983,wind,18,18159,Tipton County,county,Tipton County
778,Kansas,None,Lyon,Roads,Max tip-height Multiplier,1+ 22.86 Meters,Lyon County Wind Energy Ordinance Art. 8 Sec. 808,None,Max tip Height Multiplier + ft,22.86,Changed from 22.86 Meters to one times the WECS height plus 22.86 Meters,2008.0,2018.0,2022.0,Changed,roads,meters,173.860001,wind,20,20111,Lyon County,county,Lyon County
820,Kansas,None,Sedgwick,Banned,None,None,Sedgwick County Wind Energy Ordinance Art 3 CHapt. D Sec. pp,None,None,NaN,Changed from 60 dBA sound limit to banned,NaN,2018.0,2022.0,Changed,banned,None,NaN,wind,20,20173,Sedgwick County,county,Sedgwick County
853,Michigan,None,Huron,Structures,Meters,499.87,Huron County Wind Energy Ordinance,None,Meters,499.87,Changed from 402.336 meters to 499.87 meters,2015.0,2018.0,2022.0,Changed,structures,meters,499.869995,wind,26,26063,Huron County,county,Huron County
854,Michigan,None,Huron,Sound,dBA,45,Huron County Wind Energy Ordinance,None,dBA,45.00,Changed from 50 dBA to 45 dBA,2015.0,2018.0,2022.0,Changed,sound,dba,45.000000,wind,26,26063,Huron County,county,Huron County
1207,Nevada,None,Lyon,Height,Meters,60.96,15.336.07 Wind Energy Conversion Systems,None,Meters,60.96,Changed from 80 meters to 60.96 meters,NaN,2018.0,2022.0,None,height,meters,60.959999,wind,32,32019,Lyon County,county,Lyon County
1209,Nevada,None,Washoe,Railroads,Max tip-height Multiplier,1,Washoe County Art 326 Wind Systems,None,Max tip-height Multiplier,1.00,Changed from 3 max tip height to 1 max tip height,NaN,2018.0,2022.0,Changed,railroads,max tip height multiplier,1.000000,wind,32,32031,Washoe County,county,Washoe County


#### Fix mistaken update

In [23]:
update_item = nrel.query("raw_county_name == 'Jay' and raw_ordinance_type == 'Structures' and energy_type == 'wind'").reset_index(drop=False).squeeze()
update_item

index                                                             400
raw_state_name                                                Indiana
raw_town_name                                                    None
raw_county_name                                                   Jay
raw_ordinance_type                                         Structures
raw_units                                                      Meters
raw_value                                                         305
raw_citation                  Jay County Zoning Ordinace Sec. 217&218
raw_comment                                                      None
raw_updated_unit                                               Meters
raw_updated_value                                               457.2
raw_updated_comment           Changed from 305 Meters to 457.2 Meters
year_enacted                                                      NaN
year_recorded                                                  2018.0
updated_year_recorde

In [24]:
if update_item['raw_value'] != update_item['raw_updated_value']:
    nrel.loc[update_item['index'], "value"] = update_item['raw_updated_value']

In [25]:
nrel.query("raw_county_name == 'Jay' and raw_ordinance_type == 'Structures' and energy_type == 'wind'").reset_index(drop=False).squeeze()

index                                                             400
raw_state_name                                                Indiana
raw_town_name                                                    None
raw_county_name                                                   Jay
raw_ordinance_type                                         Structures
raw_units                                                      Meters
raw_value                                                         305
raw_citation                  Jay County Zoning Ordinace Sec. 217&218
raw_comment                                                      None
raw_updated_unit                                               Meters
raw_updated_value                                               457.2
raw_updated_comment           Changed from 305 Meters to 457.2 Meters
year_enacted                                                      NaN
year_recorded                                                  2018.0
updated_year_recorde